Telegram Bot
* здесь три ячейки для самодостаточной работы с ботом.
* сначала загружаем всё необходимое окружение, после запускаем последнюю ячейку с ботом и переходим в мессенджер.


In [ ]:
!pip install python-telegram-bot==13.3 pymorphy2 stop_words annoy num2words
!pip install -U accelerate
!pip install -U transformers

import os
import mmap
import re
from telegram.ext import Updater, CommandHandler, MessageHandler, filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import json
import num2words
from sklearn.feature_extraction.text import CountVectorizer

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

from google.colab import drive
drive.mount('/content/drive')

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gpt2small_based_tales_model/model_gpt_chf")

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))

def preprocess_txt(line):
    spls = ' '.join([re.sub(r'={3}', ' === ', i) for i in line.split()]).split()
    spls = [i for i in spls if i not in sw and i != ""]
    spls = [re.sub(r'[^а-яА-Яё=== ]', '', i) for i in spls]
    spls = [i for i in spls if len(i)>2]
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    return spls

# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

vectorizer = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("/content/drive/MyDrive/chat_bot_telegram/vectorizer.vocabulary.pkl", "rb")))

lr = pickle.load(open('/content/drive/MyDrive/chat_bot_telegram/lr_model.sav', 'rb'))

with open('/content/drive/MyDrive/chat_bot_telegram/idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)

#  Загрузим индексы #Tfidf

idfs = pd.read_pickle('/content/drive/MyDrive/chat_bot_telegram/idfs.pkl')
midf = np.mean([i for i in idfs.values()])

ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('/content/drive/MyDrive/chat_bot_telegram/speaker.ann')
index_map = pd.read_pickle("/content/drive/MyDrive/chat_bot_telegram/index_speaker.pkl")

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('/content/drive/MyDrive/chat_bot_telegram/shop.ann')
index_map_shop = pd.read_pickle("/content/drive/MyDrive/chat_bot_telegram/index_shop.pkl")

ft_index_pharm = annoy.AnnoyIndex(100 ,'angular')
ft_index_pharm.load('/content/drive/MyDrive/chat_bot_telegram/pharm.ann')
index_map_pharm = pd.read_pickle("/content/drive/MyDrive/chat_bot_telegram/index_pharm.pkl")

#загрузка FastText
modelFT = FastText.load("/content/drive/MyDrive/chat_bot_telegram/ft_model")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.3/436.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=116a8e57f9b544df0bab1b69e49383da94f700eed3ad91c3a9273bac3a410adf
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550738 sha256=4958fa815757761b7329e9d9

In [ ]:
# заменить на свой токен
updater = Updater("6315571996:AAGjw8OBxcjqGPozsN0HA-xjW_VIsQljms8", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Вас приветствует NDS_NLP_bot. Я могу поговорить с вами на свободную тему, выполнить подбор 5 наиболее подходящих товаров одежды по запросу, показать 3 лекарственных препарата по указанной вами фармакологической группе и форме выпуска, рассказать вам сказку по предложеному Вами её началу. Чего вы хотите?')

def textMessage(update, context):

    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)


    # prod_shop - товары магазина
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return

    #Tale - сказки от GPT2_small
    if prediction[0] == 2:
      #модель выбирает один из предложеных префиксов для генерации текста
      phrases_list = ['Жили были старик со старухою ', "Как то раз в один прекрасный день ", "Однажды произошла интересная история ", "У моего лучшего друга ", "Эта история о том, как я повстречал Вашу маму ", "В этот прекрасный день я встретил свою собаку", "Когда-то, давным давно, ", "Это очень грусная история о том, как", "Однажды я решил просто прогуляться и "]
      prefix = phrases_list[np.random.randint(len(phrases_list))]
      tokens = tokenizer(prefix, return_tensors='pt')
      size = tokens['input_ids'].shape[1]
      output = model1.generate(
        **tokens,
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+100,
        repetition_penalty=5.,
        temperature=0.2,
        num_beams=15,
      )

      decoded = tokenizer.decode(output[0])
      result = decoded[len(prefix):]
      context.bot.send_message(chat_id=update.message.chat_id, text=prefix + result)
      return

    #pharm - лекарственные препараты по запросу фармакологической группы и формы выпуска
    if prediction[0] == 3:
      vect_ft = embed_txt(input_txt, idfs, midf)
      ft_index_pharm_val = ft_index_pharm.get_nns_by_vector(vect_ft, 3)
      for item in ft_index_pharm_val:
          prod, country, doc = index_map_pharm[item]
          context.bot.send_message(chat_id=update.message.chat_id, text="Производитель: {} Страна производства: {} Документация препарата: {}".format(prod, country, doc))
      return

    # QA - болталка
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)

    # - уточнение вопроса при большом отклонении входного вектора
    if distances[0] > 0.5:
      print(distances[0])
      context.bot.send_message(chat_id=update.message.chat_id, text="Мне Ваш запрос непонятен. Попробуйте перефразировать.")
      return

    #ответ бота на свободную тему
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(filters.Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
